In [1]:
#Importing the required library
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import re
import joblib
import sys
import warnings
sys.modules['sklearn.externals.joblib'] = joblib
from sklearn.svm import SVR
from dateutil.relativedelta import relativedelta
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, RobustScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import VarianceThreshold
from mlxtend.feature_selection import SequentialFeatureSelector
import xgboost as xgb

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
warnings.simplefilter(action='ignore')

In [6]:
#Importing the Cleaned Lending Data
cleaned_loan_df = pd.read_csv('cleaned_lending_data.csv')
print(cleaned_loan_df.shape)
cleaned_loan_df.head()

(42535, 35)


,id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,loan_status,desc,purpose,title,addr_state,dti,inq_last_6mths,mths_since_last_delinq,open_acc,revol_bal,revol_util,total_acc,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_amnt,delinq_amnt
0,0,5000.0,5000.0,4975.0,36,10.65,162.87,B,B2,NaN,10,RENT,24000.0,Verified,0,Borrower added on 12/22/11 > I need to upgra...,credit_card,Computer,AZ,27.65,1.0,NaN,3.0,13648.0,83.70%,9.0,5863.155187,5833.84,5000.00,863.16,0.00,0.0,0.00,171.62,0.0
1,1,2500.0,2500.0,2500.0,60,15.27,59.83,C,C4,Ryder,1,RENT,30000.0,Source Verified,1,Borrower added on 12/22/11 > I plan to use t...,car,bike,GA,1.00,5.0,NaN,3.0,1687.0,9.40%,4.0,1014.530000,1014.53,456.46,435.17,0.00,122.9,1.11,119.66,0.0
2,2,2400.0,2400.0,2400.0,36,15.96,84.33,C,C5,NaN,10,RENT,12252.0,Not Verified,0,NaN,small_business,real estate business,IL,8.72,2.0,NaN,2.0,2956.0,98.50%,10.0,3005.666844,3005.67,2400.00,605.67,0.00,0.0,0.00,649.91,0.0
3,3,10000.0,10000.0,10000.0,36,13.49,339.31,C,C1,AIR RESOURCES BOARD,10,RENT,49200.0,Source Verified,0,Borrower added on 12/21/11 > to pay for prop...,other,personel,CA,20.00,1.0,35.0,10.0,5598.0,21%,37.0,12231.890000,12231.89,10000.00,2214.92,16.97,0.0,0.00,357.48,0.0
4,4,3000.0,3000.0,3000.0,60,12.69,67.79,B,B5,University Medical Group,1,RENT,80000.0,Source Verified,0,Borrower added on 12/21/11 > I plan on combi...,other,Personal,OR,17.94,0.0,38.0,15.0,27783.0,53.90%,38.0,4066.908161,4066.91,3000.00,1066.91,0.00,0.0,0.00,67.30,0.0


In [23]:
grouped_df = cleaned_loan_df['emp_title'].value_counts(normalize= True)
grouped_df

emp_title
Other                      0.915881
US Army                    0.065005
Bank of America            0.002704
IBM                        0.001693
Kaiser Permanente          0.001434
AT&T                       0.001434
UPS                        0.001364
Wells Fargo                0.001340
USAF                       0.001317
US Air Force               0.001293
Self Employed              0.001152
United States Air Force    0.001128
Walmart                    0.001105
Lockheed Martin            0.001081
State of California        0.001058
Verizon Wireless           0.001011
Name: proportion, dtype: float64

In [22]:
threshold = 0.001
# Get the list of emp_titles with low frequency
low_frequency_titles = grouped_df[grouped_df < threshold].index

# Replace the low frequency titles with 'Other'
cleaned_loan_df.loc[cleaned_loan_df['emp_title'].isin(low_frequency_titles), 'emp_title'] = 'Other'

In [25]:
from sklearn.preprocessing import LabelEncoder
# Initialize the LabelEncoder
label_encoder = LabelEncoder()
# Fit and transform the categorical variable
cleaned_loan_df['grade_Encoded'] = label_encoder.fit_transform(cleaned_loan_df['grade'])
cleaned_loan_df['sub_grade_Encoded'] = label_encoder.fit_transform(cleaned_loan_df['sub_grade'])
cleaned_loan_df.head()

,id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,loan_status,purpose,title,addr_state,dti,inq_last_6mths,mths_since_last_delinq,open_acc,revol_bal,revol_util,total_acc,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_amnt,delinq_amnt,grade_Encoded,sub_grade_Encoded
0,0,5000.0,5000.0,4975.0,36,10.65,162.87,B,B2,US Army,10,RENT,24000.0,Verified,0,credit_card,Computer,AZ,27.65,1.0,33.0,3.0,13648.0,83.70%,9.0,5863.155187,5833.84,5000.00,863.16,0.00,0.0,0.00,171.62,0.0,1,6
1,1,2500.0,2500.0,2500.0,60,15.27,59.83,C,C4,Other,1,RENT,30000.0,Source Verified,1,car,bike,GA,1.00,5.0,33.0,3.0,1687.0,9.40%,4.0,1014.530000,1014.53,456.46,435.17,0.00,122.9,1.11,119.66,0.0,2,13
2,2,2400.0,2400.0,2400.0,36,15.96,84.33,C,C5,US Army,10,RENT,12252.0,Not Verified,0,small_business,real estate business,IL,8.72,2.0,33.0,2.0,2956.0,98.50%,10.0,3005.666844,3005.67,2400.00,605.67,0.00,0.0,0.00,649.91,0.0,2,14
3,3,10000.0,10000.0,10000.0,36,13.49,339.31,C,C1,Other,10,RENT,49200.0,Source Verified,0,other,personel,CA,20.00,1.0,35.0,10.0,5598.0,21%,37.0,12231.890000,12231.89,10000.00,2214.92,16.97,0.0,0.00,357.48,0.0,2,10
4,4,3000.0,3000.0,3000.0,60,12.69,67.79,B,B5,Other,1,RENT,80000.0,Source Verified,0,other,Personal,OR,17.94,0.0,38.0,15.0,27783.0,53.90%,38.0,4066.908161,4066.91,3000.00,1066.91,0.00,0.0,0.00,67.30,0.0,1,9


In [18]:
grouped_df = cleaned_loan_df['title'].value_counts(normalize= True)
grouped_df

title
Debt Consolidation                                                                  0.053415
Debt Consolidation Loan                                                             0.041378
Personal Loan                                                                       0.016645
Consolidation                                                                       0.012860
debt consolidation                                                                  0.012507
Home Improvement                                                                    0.008769
Credit Card Consolidation                                                           0.008699
Debt consolidation                                                                  0.008158
Small Business Loan                                                                 0.007829
Personal                                                                            0.007758
Credit Card Loan                                                

In [19]:
grouped_df = cleaned_loan_df['grade'].value_counts(normalize= True)
grouped_df

grade
B    0.291266
A    0.239403
C    0.205478
D    0.141436
E    0.079793
F    0.030587
G    0.012037
Name: proportion, dtype: float64

In [7]:
cleaned_loan_df.drop('desc',axis = 1,inplace = True)
cleaned_loan_df.shape

(42535, 34)

In [ ]:
def impute_missing_values(df):
    for column in df.columns:
        if df[column].dtype == object:  # Categorical feature
            mode_value = df[column].mode()[0]
            df[column].fillna(mode_value, inplace=True)
        else:  # Numerical feature
            median_value = df[column].median()
            df[column].fillna(median_value, inplace=True)
    return df
impute_missing_values(cleaned_loan_df)

In [ ]:
#extract the numeric part from the variable revol_util

# using regular expression to extract numeric values from the string
cleaned_loan_df['revol_util'] = cleaned_loan_df['revol_util'].apply(lambda x: re.findall('\d+', str(x))[0])
# convert to numeric
cleaned_loan_df['revol_util'] = cleaned_loan_df['revol_util'].apply(lambda x: pd.to_numeric(x))

In [ ]:
#One-Hot Encoding for Categorical Feature
cat_cols = categorical_columns.copy()
cleaned_loan_one_hot_df = cleaned_loan_df.copy()
cat_cols.remove('term')
cat_cols.remove('loan_status')
for i in cat_cols:
    one_hot_encoded = pd.get_dummies(cleaned_loan_one_hot_df[i] ,prefix=None,drop_first = True)
    cleaned_loan_one_hot_df = pd.concat([cleaned_loan_one_hot_df, one_hot_encoded], axis=1)
    cleaned_loan_one_hot_df.drop(i, axis=1, inplace=True)
print(cleaned_loan_one_hot_df.shape)
print(cleaned_loan_one_hot_df.columns)
cleaned_loan_one_hot_df.head(10)

In [ ]:
print(cleaned_loan_one_hot_df.shape)
print(cleaned_loan_one_hot_df.columns)
cleaned_loan_one_hot_df.head(10)

In [ ]:
target_var = 'loan_status'
id_var = 'id'
X = cleaned_loan_df.drop([target_var,id_var], axis=1)
y = cleaned_loan_df[target_var]

In [ ]:
def Spliting_Data_Test_Train(X, y):

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)
    return X_train, X_test, y_train, y_test


X_train, X_test, y_train, y_test = Spliting_Data_Test_Train(X, y)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

In [ ]:
#Feature selection
# Create a sequential forward selector with Random Forest Regressor
sfs = SequentialFeatureSelector(RandomForestRegressor(),
                                k_features=5,  # Number of features to select
                                forward=True,  # Forward selection
                                floating=False,  # Disable floating feature selection
                                scoring='r2',  # Evaluation criterion
                                cv=0)  # No cross-validation

# Perform feature selection
sfs.fit(X, y)

# Selected feature indices
selected_feature_indices = sfs.k_feature_idx_
print("Selected Feature Indices:", selected_feature_indices)

# Selected feature names
selected_feature_names = sfs.k_feature_names_
print("Selected Feature Names:", selected_feature_names)